In [2]:
import numpy as np
import torch
import torch.utils.data
from connectomics.data.dataset import VolumeDatasetCond
from connectomics.data.utils import readh5, syn_sem2inst, seg2polarity
from connectomics.data.dataset.collate import collate_fn_train

In [2]:
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

def show(image, cmap='viridis', title='Test Title', interpolation=None):  
    num_imgs = image.shape[-3] # c (optional), z, y, x
    fig = plt.figure(figsize=(20., 3.))
    fig.suptitle(title, fontsize=14)
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(1, num_imgs),  # creates 2x2 grid of axes
                     axes_pad=0.1,  # pad between axes in inch.
                     )
    image_list = np.split(image, num_imgs, -3)
    for ax, im in zip(grid, [np.squeeze(x) for x in image_list]):
        # Iterating over the grid returns the Axes.
        if im.ndim == 3:
            im = im.transpose(1,2,0)
        ax.imshow(im, cmap=cmap, interpolation=interpolation)
        ax.axis('off')

    plt.show()

In [7]:
# load volumes
# img = readh5("../../../datasets/cremi/sample_A_20160501.hdf", "volumes/raw")
# img = readh5("../../../datasets/cremi/sample_A_20160501.hdf", "volumes/labels")
# img = readh5("../../../datasets/cremi/sample_A_20160501.hdf", "volumes/labels/clefts")
# img = readh5("../../../datasets/cremi/sample_A_20160501.hdf", "volumes/labels/neuron_ids")
# img = readh5("../../../datasets/cremi/sample_A_20160501.hdf", "volumes")


img = readh5("../../../datasets/cremi_edited/labels/label_sample_A_20160501.h5", "labels")
# img = readh5("../../../datasets/cremi_edited/labels/label_sample_B_20160501.h5", "labels")

# img = readh5("../outputs/CREMI_Binary_UNet/test/result_0_txy.h5", "vol0")

# img = readh5("../../../datasets/cremi/sample_A_padded_20160501.hdf")

# syn = readh5("../datasets/synapse/syn.h5")
# syn = syn_sem2inst(syn)
print(img.shape, img.dtype)
# print(syn.shape, syn.dtype)

# polarity = seg2polarity(syn)
# show(img[50:60,:400,:400], title="EM Images", cmap='gray')
# show(polarity[:,50:60,:400,:400], title="Synaptic Polarity Masks")

(125, 1250, 1250) uint64


In [8]:
print(img)

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


In [9]:
uid = np.unique(img)

In [10]:
max(uid)


884

In [2]:
%load_ext tensorboard
import tensorflow as tf
import datetime

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


2023-06-13 17:11:13.855146: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-13 17:11:15.936573: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-13 17:11:19.201122: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-13 17:11:19.202034: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 17:11:35.497971: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
%tensorboard --logdir /home/suryakalia/documents/summer/exploration/pytorch_connectomics/outputs/CREMI_Binary_UNet/